In [1]:
pip install faiss

     |████████████████████████████████| 4.7 MB 4.3 MB/s 


In [2]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (125 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [3]:
!pip3 install pickle5
import pickle5 as pickle

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# DATA_PATH = "gdrive/MyDrive"

# with open(DATA_PATH +'/mbert_chi_embeddings_229730.pkl','rb') as f:
#   embeddings = pickle.load(f)

In [4]:
DATA_PATH = "gdrive/MyDrive"

with open(DATA_PATH +'/bert_translated_229730.pkl','rb') as f:
  embeddings = pickle.load(f)

In [5]:
print(len(embeddings["chi_ques"]))
print(len(embeddings["chi_ans"]))
print(len(embeddings["translated_ques"]))
print(len(embeddings["translated_ans"]))
print(len(embeddings["ques_embeddings"]))
print(len(embeddings["ans_embeddings"]))

229730
229730
229730
229730
229730
229730


In [6]:
import numpy as np 
import faiss

In [7]:
question_embedding = np.array(embeddings["ques_embeddings"])
answer_embedding = np.array(embeddings["ans_embeddings"])
print(question_embedding.shape)
print(answer_embedding.shape)

(229730, 768)
(229730, 768)


In [8]:
dimension = 768
nlist = 5          # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  
index_question = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)
index_answer = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

In [9]:
print(index_question.is_trained)   # False
index_question.train(question_embedding)  # train on the database vectors
print(index_question.ntotal)   # 0
index_question.add(question_embedding)   # add the vectors and update the index
print(index_question.is_trained)  # True
print(index_question.ntotal)

False
0
True
229730


In [10]:
print(index_answer.is_trained)   # False
index_answer.train(answer_embedding)  # train on the database vectors
print(index_answer.ntotal)   # 0
index_answer.add(answer_embedding)   # add the vectors and update the index
print(index_answer.is_trained)  # True
print(index_answer.ntotal)   # 29752

False
0
True
229730


In [11]:
embedding_by_len = embeddings["chi_ques"].copy()
embedding_by_len.sort(key=len, reverse=True)

n = 15000

index = []

for i in range(n):
  index.append(embeddings["chi_ques"].index(embedding_by_len[i]))

In [12]:
len(index)

15000

In [13]:
k = 4

result = []

for i in range(n):
  num = index[i]
  question_query_vector = np.array([embeddings["ques_embeddings"][num]])
  question_distances, question_indices = index_question.search(question_query_vector, k)
  
  answer_query_vector = np.array([embeddings["ans_embeddings"][num]])
  answer_distances, answer_indices = index_answer.search(answer_query_vector, k)
  
  for j in range(k):
    q_index = question_indices[0][j]
    if (q_index != num) and q_index in answer_indices[0]:
      if embeddings["chi_ques"][num] != embeddings["chi_ques"][q_index] and embeddings["chi_ans"][num] != embeddings["chi_ans"][q_index]:
        result.append((num, q_index))

In [16]:
print(result)

[(132470, 50565), (115836, 77898), (46155, 123791), (134981, 146055), (115845, 115848), (115845, 220466), (165429, 223460), (102572, 71278), (102572, 102407), (102572, 53863), (173193, 119473), (173193, 4241), (135388, 92902), (133460, 224725), (14147, 103276), (109853, 46518), (78793, 215973), (78793, 151971), (100188, 1782), (100188, 10083), (100308, 182116), (117307, 30085), (117307, 115435), (67761, 228388), (67761, 22901), (87533, 48607), (92778, 227250), (92778, 205373), (2853, 62102), (2853, 207905), (30897, 53159), (146055, 134981), (218046, 139757), (108066, 85706), (108066, 52768), (75274, 127204), (75274, 93423), (127204, 75274), (127204, 93423), (169074, 192494), (169074, 216951), (54871, 21136), (54871, 228120), (134553, 96106), (121782, 33292), (120603, 76448), (120603, 220366), (101257, 92785), (102520, 185083), (111501, 173472), (111501, 26243), (157408, 116922), (131007, 1920), (131007, 47700), (220140, 226769), (220140, 28243), (187481, 227215), (50398, 132868), (1888

In [21]:
print(len(result))

9536


In [22]:
print(embeddings["translated_ques"][132470])
print(embeddings["translated_ans"][132470])

Fear of HIV/AIDS.Good evening, chief! Disturbing your marriage! I'm in pain! There are too many symptoms! 18 years of massage, bathing, massage, sucking my nipples, leaving a lot of saliva (not knowing if there's blood in the saliva), and I touched it a few times, and my hand's blistering position (the kind that can squeeze a little bit of blood, about 3 mm size) was under the no-name finger, and now I think it's the woman mechanic who threw up a lot of water (not knowing if there's blood in the saliva) on my nipples, and then I touched it a few times.After that, and after that, I discovered that the massage girl's neck was ringing and my neck was ringing for some time, and I was afraid that she was contagious in the window. Five days after the massage, I had a little pain in the back of my ear, and then two weeks later, I had a nausea and vomiting feeling (which lasted 10 days or so), three weeks with low blood lymphocytes, about three weeks of low lymph lymph lymph lymph lymph lymph 

In [23]:
print(embeddings["translated_ques"][50565])
print(embeddings["translated_ans"][50565])

Good morning, Director Zhang! Thank you!My mom and dad did the whole pulmonary test. I'll see you later.Chief Zhang, what do you need to do with the lung pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary pulmonary tests?
 The pulmonary knot is a very high-density micropulmonary knot, which is too small to perform further tests for CT-guided pulmonary punctures and is more likely to take into account benignness.A review of the CT comparison will be available in three to six months.


In [18]:
for i in range(len(result)):
  first_index = result[i][0]
  second_index = result[i][1]

  first_q = embeddings["chi_ques"][first_index]
  first_a = embeddings["chi_ans"][second_index]
  first_t_q = embeddings["translated_ques"][first_index]
  first_t_a =embeddings["translated_ans"][second_index]
  first_q_embeddings = embeddings["ques_embeddings"][first_index]
  first_a_embeddings = embeddings["ans_embeddings"][second_index]

  second_q = embeddings["chi_ques"][second_index]
  second_a = embeddings["chi_ans"][first_index]
  second_t_q = embeddings["translated_ques"][second_index]
  second_t_a =embeddings["translated_ans"][first_index]
  second_q_embeddings = embeddings["ques_embeddings"][second_index]
  second_a_embeddings = embeddings["ans_embeddings"][first_index]

  embeddings["chi_ques"].append(first_q)
  embeddings["chi_ans"].append(first_a)
  embeddings["translated_ques"].append(first_t_q)
  embeddings["translated_ans"].append(first_t_a)
  embeddings["ques_embeddings"].append(first_q_embeddings)
  embeddings["ans_embeddings"].append(first_a_embeddings)

  embeddings["chi_ques"].append(second_q)
  embeddings["chi_ans"].append(second_a)
  embeddings["translated_ques"].append(second_t_q)
  embeddings["translated_ans"].append(second_t_a)
  embeddings["ques_embeddings"].append(second_q_embeddings)
  embeddings["ans_embeddings"].append(second_a_embeddings)

In [19]:
print(len(embeddings["chi_ques"]))
print(len(embeddings["chi_ans"]))
print(len(embeddings["translated_ques"]))
print(len(embeddings["translated_ans"]))
print(len(embeddings["ques_embeddings"]))
print(len(embeddings["ans_embeddings"]))

248802
248802
248802
248802
248802
248802


In [20]:
pickle_out = open(DATA_PATH + "/ MedDialog_Translated_BERT.pkl", "wb")
pickle.dump(embeddings, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()